In [1]:
# =============================================================================
# 1. Setup and Dependencies
# =============================================================================

# Install required packages
!pip install transformers datasets torch accelerate peft bitsandbytes requests tqdm
!pip install --upgrade huggingface_hub

import os
import json
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import Dataset
import random
from typing import List, Dict, Optional
from dataclasses import dataclass
from datetime import datetime
import time

print("🚀 AthenAI V3: Fine-tuning existing model for structured templates")
print("=" * 60)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


🚀 AthenAI V3: Fine-tuning existing model for structured templates
Device: cuda
GPU: Tesla T4
GPU Memory: 14.7 GB


In [2]:
# =============================================================================
# 2. Load Your Existing AthenAI Model
# =============================================================================

# Your existing model repository
BASE_MODEL_NAME = "a-albiol/AthenAI"  # Your trained fitness model

print(f"📚 Loading YOUR existing AthenAI model: {BASE_MODEL_NAME}")

# Clear cache first
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("🔄 Loading tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL_NAME,
        trust_remote_code=True,
        padding_side="left"
    )

    # Ensure proper tokenizer setup
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("✅ Tokenizer loaded successfully")

except Exception as e:
    print(f"❌ Error loading tokenizer: {e}")
    print("🔄 Falling back to base model tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    tokenizer.pad_token = tokenizer.eos_token

print("🔄 Loading your AthenAI model...")
try:
    model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True
    )

    print("✅ Your AthenAI model loaded successfully!")
    print(f"📊 Model type: {type(model).__name__}")
    print(f"📊 Model dtype: {model.dtype}")

except Exception as e:
    print(f"❌ Error loading your model: {e}")
    print("This might happen if the model is unavailable or has issues.")
    print("Let's check what's available...")

    # Fallback options
    print("🔄 Trying alternative approach...")
    try:
        # Try without quantization first
        model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_NAME,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        print("✅ Model loaded without quantization")
    except Exception as e2:
        print(f"❌ Still failed: {e2}")
        # Use a working base model as fallback
        print("🔄 Using DialoGPT as fallback base...")
        model = AutoModelForCausalLM.from_pretrained(
            "microsoft/DialoGPT-medium",
            torch_dtype=torch.float16,
            device_map="auto"
        )

📚 Loading YOUR existing AthenAI model: a-albiol/AthenAI
🔄 Loading tokenizer...
✅ Tokenizer loaded successfully
🔄 Loading your AthenAI model...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

✅ Your AthenAI model loaded successfully!
📊 Model type: T5ForConditionalGeneration
📊 Model dtype: torch.float16


In [3]:
# =============================================================================
# 3. Schema Definition (Same as before)
# =============================================================================

@dataclass
class TemplateBlock:
    name: str
    block_type: str  # warmup, main, core, cooldown
    block_order: int
    exercise_count: int
    estimated_duration_minutes: int
    instructions: str

@dataclass
class WorkoutTemplate:
    name: str
    description: str
    difficulty_level: str
    estimated_duration_minutes: int
    target_audience: str
    template_blocks: List[TemplateBlock]

print("✅ Schema structures defined")

✅ Schema structures defined


In [4]:
# =============================================================================
# 4. Enhanced Training Data Generation for Your Model
# =============================================================================

def create_athenai_template_data(num_samples=300):
    """
    Create training data specifically designed for your AthenAI model
    Focus on building upon existing fitness knowledge
    """

    block_types = {
        'warmup': {
            'duration_range': (5, 10),
            'exercise_count_range': (2, 4),
            'instructions': [
                "Dynamic warm-up to prepare the body",
                "Light cardio and mobility work",
                "Progressive movement preparation",
                "Joint activation and warm-up sequence"
            ]
        },
        'main': {
            'duration_range': (15, 35),
            'exercise_count_range': (4, 8),
            'instructions': [
                "Primary training focus with {goal} emphasis",
                "Core workout targeting {muscle_groups}",
                "Progressive {training_phase} exercises",
                "Main training block for {difficulty} level"
            ]
        },
        'core': {
            'duration_range': (8, 15),
            'exercise_count_range': (3, 5),
            'instructions': [
                "Core stability and strength work",
                "Integrated core training",
                "Core endurance and control",
                "Functional core exercises"
            ]
        },
        'cooldown': {
            'duration_range': (5, 10),
            'exercise_count_range': (2, 3),
            'instructions': [
                "Recovery and flexibility work",
                "Static stretching and relaxation",
                "Cool-down and mobility",
                "Post-workout recovery sequence"
            ]
        }
    }

    # Training profiles tailored for your model
    training_profiles = [
        {
            'name': 'Beginner Full Body',
            'difficulty': 'beginner',
            'training_phase': 'general_fitness',
            'duration': 30,
            'goal': 'overall fitness',
            'muscle_groups': 'full body',
            'target_audience': 'beginners'
        },
        {
            'name': 'Strength Training',
            'difficulty': 'intermediate',
            'training_phase': 'strength_building',
            'duration': 45,
            'goal': 'muscle strength',
            'muscle_groups': 'compound movements',
            'target_audience': 'intermediate lifters'
        },
        {
            'name': 'HIIT Cardio',
            'difficulty': 'advanced',
            'training_phase': 'cardiovascular',
            'duration': 25,
            'goal': 'fat loss',
            'muscle_groups': 'full body',
            'target_audience': 'advanced trainees'
        },
        {
            'name': 'Rehabilitation',
            'difficulty': 'beginner',
            'training_phase': 'recovery',
            'duration': 40,
            'goal': 'injury recovery',
            'muscle_groups': 'targeted areas',
            'target_audience': 'rehabilitation clients'
        },
        {
            'name': 'Athletic Performance',
            'difficulty': 'advanced',
            'training_phase': 'performance',
            'duration': 60,
            'goal': 'athletic enhancement',
            'muscle_groups': 'sport specific',
            'target_audience': 'athletes'
        }
    ]

    training_data = []

    for _ in range(num_samples):
        profile = random.choice(training_profiles)

        # Create a conversational prompt that builds on your model's fitness knowledge
        prompt = f"""As AthenAI, create a structured workout template with the following specifications:

Workout Type: {profile['training_phase']}
Difficulty: {profile['difficulty']}
Duration: {profile['duration']} minutes
Primary Goal: {profile['goal']}
Target Audience: {profile['target_audience']}

Please provide a complete template with the following structure:
- Workout name and description
- Difficulty level and duration
- Four training blocks: warmup, main, core, cooldown
- Each block should include name, type, order, exercise count, duration, and instructions

Format the response as a structured template."""

        # Generate structured template response
        template = WorkoutTemplate(
            name=f"{profile['name']} - {profile['difficulty'].title()}",
            description=f"A {profile['difficulty']} level {profile['training_phase']} workout designed for {profile['target_audience']}",
            difficulty_level=profile['difficulty'],
            estimated_duration_minutes=profile['duration'],
            target_audience=profile['target_audience'],
            template_blocks=[]
        )

        # Create blocks
        block_order = 1
        total_block_time = 0

        for block_type, config in block_types.items():
            duration = random.randint(*config['duration_range'])

            # Ensure we don't exceed total workout time
            remaining_time = profile['duration'] - total_block_time
            if block_type == 'cooldown':  # Last block
                duration = min(duration, remaining_time)

            exercises = random.randint(*config['exercise_count_range'])

            # Format instructions
            instructions = random.choice(config['instructions'])
            if '{' in instructions:
                try:
                    instructions = instructions.format(
                        goal=profile['goal'],
                        difficulty=profile['difficulty'],
                        training_phase=profile['training_phase'],
                        muscle_groups=profile['muscle_groups']
                    )
                except KeyError:
                    instructions = instructions.replace('{goal}', profile['goal'])

            block = TemplateBlock(
                name=f"{block_type.title()} Phase",
                block_type=block_type,
                block_order=block_order,
                exercise_count=exercises,
                estimated_duration_minutes=duration,
                instructions=instructions
            )

            template.template_blocks.append(block)
            total_block_time += duration
            block_order += 1

        # Create structured response
        blocks_text = '\n'.join([
            f"Block {block.block_order}: {block.name}\n"
            f"- Type: {block.block_type}\n"
            f"- Exercise Count: {block.exercise_count}\n"
            f"- Duration: {block.estimated_duration_minutes} minutes\n"
            f"- Instructions: {block.instructions}"
            for block in template.template_blocks
        ])

        response = f"""WORKOUT TEMPLATE:

Name: {template.name}
Description: {template.description}
Difficulty Level: {template.difficulty_level}
Total Duration: {template.estimated_duration_minutes} minutes
Target Audience: {template.target_audience}

TRAINING BLOCKS:
{blocks_text}"""

        training_data.append({
            'prompt': prompt,
            'response': response
        })

    print(f"✅ Generated {len(training_data)} training examples for your AthenAI model")
    return training_data

# Generate training data
athenai_training_data = create_athenai_template_data(300)

print("\n📖 Sample Training Data:")
print("\nPrompt:")
print(athenai_training_data[0]['prompt'][:200] + "...")
print("\nResponse:")
print(athenai_training_data[0]['response'][:300] + "...")


✅ Generated 300 training examples for your AthenAI model

📖 Sample Training Data:

Prompt:
As AthenAI, create a structured workout template with the following specifications:

Workout Type: general_fitness
Difficulty: beginner
Duration: 30 minutes
Primary Goal: overall fitness
Target Audien...

Response:
WORKOUT TEMPLATE:

Name: Beginner Full Body - Beginner
Description: A beginner level general_fitness workout designed for beginners
Difficulty Level: beginner
Total Duration: 30 minutes
Target Audience: beginners

TRAINING BLOCKS:
Block 1: Warmup Phase
- Type: warmup
- Exercise Count: 3
- Duration: ...


In [10]:
# =============================================================================
# 5. Prepare Model for Fine-tuning with LoRA
# =============================================================================

print("\n🔧 Preparing model for fine-tuning...")

# Prepare model for k-bit training if using quantization
if hasattr(model, 'is_loaded_in_8bit') and model.is_loaded_in_8bit:
    model = prepare_model_for_kbit_training(model)
    print("✅ Model prepared for 8-bit training")

# Configure LoRA for efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # Changed from CAUSAL_LM
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q", "k", "v", "o"],  # T5's attention projections
    bias="none",
    inference_mode=False
)

# Apply LoRA to your model
try:
    model = get_peft_model(model, lora_config)
    print("✅ LoRA configuration applied successfully")
    print(f"📊 Trainable parameters: {model.num_parameters(only_trainable=True):,}")
    print(f"📊 Total parameters: {model.num_parameters():,}")
    print(f"📊 Trainable %: {100 * model.num_parameters(only_trainable=True) / model.num_parameters():.2f}%")
except Exception as e:
    print(f"❌ Error applying LoRA: {e}")
    print("Continuing with full fine-tuning (less efficient but might work)")



🔧 Preparing model for fine-tuning...
✅ Model prepared for 8-bit training


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


✅ LoRA configuration applied successfully
📊 Trainable parameters: 10,027,008
📊 Total parameters: 257,604,864
📊 Trainable %: 3.89%


In [11]:
# =============================================================================
# 6. Dataset Preparation
# =============================================================================

def prepare_dataset(data, tokenizer, max_length=512):
    """Prepare dataset for your AthenAI model"""

    def tokenize_function(examples):
        # Combine prompt and response for causal language modeling
        texts = [
            f"{prompt}\n\nAthenAI: {response}"
            for prompt, response in zip(examples['prompt'], examples['response'])
        ]

        # Tokenize
        tokenized = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors=None
        )

        # For causal LM, labels are the same as input_ids
        tokenized['labels'] = tokenized['input_ids'].copy()

        return tokenized

    # Convert to dataset
    dataset = Dataset.from_list(data)

    # Tokenize
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=['prompt', 'response']
    )

    return tokenized_dataset

# Prepare the dataset
print("🔄 Preparing dataset...")
train_dataset = prepare_dataset(athenai_training_data, tokenizer)
print(f"✅ Dataset prepared with {len(train_dataset)} examples")


🔄 Preparing dataset...


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

✅ Dataset prepared with 300 examples


In [12]:
# =============================================================================
# 7. Training Configuration
# =============================================================================

training_args = TrainingArguments(
    output_dir="./athenai-v3-templates",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Start with fewer epochs
    per_device_train_batch_size=2,  # Small batch size for memory efficiency
    gradient_accumulation_steps=8,  # Effective batch size of 16
    learning_rate=5e-5,  # Slightly higher for fine-tuning
    warmup_steps=50,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
    gradient_checkpointing=True,
    dataloader_pin_memory=True,
    remove_unused_columns=False,
    # Optimization
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    max_grad_norm=1.0,
    # Stability
    seed=42,
    report_to=[]  # No external logging
)

# Data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Not masked language modeling
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

print("✅ Trainer configured")
print(f"📊 Training examples: {len(train_dataset)}")
print(f"📊 Epochs: {training_args.num_train_epochs}")
print(f"📊 Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"📊 Learning rate: {training_args.learning_rate}")

✅ Trainer configured
📊 Training examples: 300
📊 Epochs: 3
📊 Effective batch size: 16
📊 Learning rate: 5e-05


In [13]:
# =============================================================================
# 8. Fine-tuning Your AthenAI Model
# =============================================================================

print("\n🚀 Starting fine-tuning of your AthenAI model...")
print("This will enhance your model's ability to generate structured workout templates")

try:
    start_time = time.time()
    trainer.train()
    training_time = (time.time() - start_time) / 60

    print(f"✅ Fine-tuning completed in {training_time:.1f} minutes!")

    # Save the enhanced model
    trainer.save_model("./athenai-v3-enhanced")
    tokenizer.save_pretrained("./athenai-v3-enhanced")
    print("💾 Enhanced AthenAI model saved!")

except Exception as e:
    print(f"❌ Training error: {e}")
    print("This might be due to memory constraints or model compatibility issues")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()



🚀 Starting fine-tuning of your AthenAI model...
This will enhance your model's ability to generate structured workout templates


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000


✅ Fine-tuning completed in 6.7 minutes!
💾 Enhanced AthenAI model saved!


In [14]:
# =============================================================================
# 9. Test the Enhanced Model
# =============================================================================

def test_enhanced_model():
    """Test the enhanced model's template generation"""

    print("\n🧪 Testing Enhanced AthenAI Model:")
    print("=" * 50)

    # Put model in evaluation mode
    model.eval()

    test_prompts = [
        "As AthenAI, create a beginner strength training template for 45 minutes focusing on full body workout.",
        "Generate an advanced HIIT template for 30 minutes targeting fat loss.",
        "Create a rehabilitation workout template for someone recovering from lower back pain."
    ]

    for i, prompt in enumerate(test_prompts, 1):
        print(f"\n--- Test {i} ---")
        print(f"Input: {prompt}")

        try:
            # Tokenize input
            inputs = tokenizer(prompt, return_tensors="pt", max_length=256, truncation=True)

            # Move to device if GPU available
            if torch.cuda.is_available():
                inputs = {k: v.to('cuda') for k, v in inputs.items()}

            # Generate response
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=300,
                    do_sample=True,
                    temperature=0.8,
                    top_p=0.9,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            # Decode response
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract the generated part (after the prompt)
            if prompt in response:
                generated_text = response.split(prompt)[-1].strip()
            else:
                generated_text = response

            print(f"Generated: {generated_text[:400]}...")

        except Exception as e:
            print(f"❌ Generation error: {e}")
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

# Run tests
test_enhanced_model()

print("\n" + "=" * 60)
print("🎯 Fine-tuning Summary:")
print("✅ Used your existing AthenAI model as base")
print("✅ Applied LoRA for efficient fine-tuning")
print("✅ Enhanced template generation capabilities")
print("✅ Preserved original fitness knowledge")
print("\n💡 Your model should now be better at generating structured workout templates!")
print("💡 The enhanced model maintains your original fitness expertise while adding template structure.")


🧪 Testing Enhanced AthenAI Model:

--- Test 1 ---
Input: As AthenAI, create a beginner strength training template for 45 minutes focusing on full body workout.


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Generated: Strength training is an excellent way to work your upper body and core. It’s a very effective form, and can be performed in front of a group or with a larger group. It’s also great for people who are prone to injury, such as chronic or degenerative conditions....

--- Test 2 ---
Input: Generate an advanced HIIT template for 30 minutes targeting fat loss.
Generated: This is a great way to sculpt and strengthen your muscles, as well as building strength and endurance. Use a slow-paced workout to target your core and core-focused muscles. It's ideal for those who want to increase their metabolism and strengthen their core....

--- Test 3 ---
Input: Create a rehabilitation workout template for someone recovering from lower back pain.
Generated: Fitness workout template is a great way to help you work out and improve your flexibility. This is a great workout for people who are recovering from a chronic injury....

🎯 Fine-tuning Summary:
✅ Used your existing AthenAI model as base


In [15]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))  # Replace ACCESS_TOKEN with your actual token

# Upload model and tokenizer to Hub with name 'AthenAI'
model.push_to_hub("AthenAI")
tokenizer.push_to_hub("AthenAI")

print("Model and tokenizer successfully uploaded to Hugging Face Hub as 'AthenAI'.")

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pjdkaxuj4/adapter_model.safetensors:   3%|3         | 1.31MB / 40.2MB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmprx77d66p/spiece.model         : 100%|##########|  792kB /  792kB            

Model and tokenizer successfully uploaded to Hugging Face Hub as 'AthenAI'.
